In [20]:
import os
from glob import glob

import torch
import torchaudio as ta       

from model.demucs import Demucs
from omegaconf import OmegaConf

import librosa
import librosa.display as lds
import IPython.display as ipd


In [8]:
torch_model_path = '../results/Demucs_pl_ru_ukr/checkpoints/torch_214.ckpt'
config_path = './config.yaml'

In [9]:
checkpoint = torch.load(torch_model_path)
config = OmegaConf.load(config_path)
model = Demucs(**config.demucs)
model.load_state_dict(checkpoint)

<All keys matched successfully>

In [11]:
def enhance(model, noisy_mix, sample_len):
    if noisy_mix.size(-1) % sample_len != 0:
        padded_length = sample_len - (
            noisy_mix.size(-1) % sample_len
        )
        noisy_mix = torch.cat(
            [noisy_mix,torch.zeros(size=(1, 1, padded_length))],
            dim=-1)

    assert (noisy_mix.size(-1) % sample_len == 0 and noisy_mix.dim() == 3)

    noisy_chunks = list(torch.split(noisy_mix, sample_len, dim=-1))

    # Get enhanced full audio
    enhanced_chunks = [model(chunk).detach().cpu() for chunk in noisy_chunks]
    enhanced = torch.cat(enhanced_chunks, dim=-1)

    if padded_length != 0:
        enhanced = enhanced[:, :, :-padded_length]
        noisy_mix = noisy_mix[:, :, :-padded_length]

    enhanced = enhanced.reshape(-1).detach().cpu().numpy()
    return enhanced

In [26]:
ls

README.md*    dataset/         lightning_logs/           pl2torch.py
__pycache__/  enhancement.py*  model/                    train_pl.py*
augment.py*   git_push.sh*     model_wrap_pl.py*         utils.py*
config.yaml*  inference.ipynb  noisy_records_generator/


In [34]:
with open('../val.txt', 'r') as f:
    d_set = f.readlines()
    
files = [f.strip('\n').split(' ')[1] for f in d_set]

In [35]:
signal, sr = ta.load(files[0])
ipd.Audio(signal, rate=sr)

In [39]:
out = enhance(model, signal.unsqueeze(0), config.sample_len)

In [40]:
ipd.Audio(out, rate=sr)